# topic modeling pipeline

This notebook will walk you through:
1. data reterival from SQL database
2. Pre-processing data
3. Running topic modeling (different models)
4. post processing (including showing topics, topic-doc distribution, etc.)
5. Storing results into DB

# imports

In [1]:
from pre_processing import remove_html_tags, preprocess_data, prepare_corpus
from rpy2 import robjects #loading R inside Python
#importing functions to read pandas DF
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

import pandas as pd

import numpy as np

#import R base library
base = importr('base')

mallet_path = 'mallet' #'mallet for mac-os

[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Loading data from SQL database

First creating a connection

**In order to create a connection from anywhere in LU network you need to run the command below in terminal. Replace your username with amh418 in the command below.**

ssh -L 3306:127.0.0.1:3306 -N -f amh418@128.180.111.76   #it will ask for lehigh pass

on network (128.180.111.76)
outside the network (ssh.lehigh.edu)

**if you need to kill to SSH forwarding use the link below**

https://superuser.com/questions/87014/how-do-i-remove-an-ssh-forwarded-port


In [2]:
import mysql.connector

#connecting to DB
cnx = mysql.connector.connect(user='push', password='ElI)g!LDOBIwM)%*5o5{dbfz('
                              ,host='127.0.0.1',database='edml')

Retreiving all the posts

In [3]:
cursor = cnx.cursor() #making a cursor to execute a SQL command
cursor.execute("select * from post") #execute a command
myresult = cursor.fetchall() #get all the results 
print('{0} posts were retreived'.format(len(myresult))) 

1129 posts were retreived


# preprocessing the data

First, removing html tags

prepare the corupus

In [4]:
#removing tags from texts
proc_texts = [remove_html_tags(i[3]) for i in myresult]
#pre_processing and tokenizing docs
pre_processed_docs,orig_docs = preprocess_data(proc_texts)
#attaching (id,original_text,pre_processed_text) together
id_text = [(myresult[orig_docs[i][1]][0],orig_docs[i][0],pre_processed_docs[i]) for i in range(len(orig_docs))]

# vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs) ==> we don't need prepare corpus as we are passing this to LDA and STM R and there they will do tokenization 

#creat a pandas DF with one column as text, one column as original text, one column as SQL post-id, etc.
text_df = pd.DataFrame(zip([i[0] for i in id_text],[' '.join(i[2]) for i in id_text]),
                       columns=['id','text'])

# Running topic modeling (with or without evaluation)

In [5]:
is_eval = False #not evaluating the model
#need to provide number of topics and other specification
model_spec = {'name':'LDA','K':10,'epochs':2000,'burnin_iteration':20,'after_iteration_burnin':50,
             'save_flag':True,'save_path':'LDA_test'}
#model_spec = {'name':'STM','K':10,'epochs':500,'prevalence':'','content':'',pre_trained_model = 'LDA',
                # 'burnin_iteration':20,'LDA_epochs':50,'save_flag':True

if model_spec['name'] == 'LDA':
    if is_eval:
        pass
    else: 
        robjects.r.source('LDA_R.R')
        ans = robjects.r.findTopics(docs = text_df,n_topics = model_spec['K'],epochs = model_spec['epochs'],
                                    burnin_iteration = model_spec['burnin_iteration'],
                                    after_iteration_burnin =  model_spec['after_iteration_burnin'],
                                    save_flag=model_spec['save_flag'],save_path = model_spec['save_path'])
        top_terms = np.asarray(ans[1])
        topic_doc_dist = np.asarray(ans[0][0]).T
        term_probab = ans[0][1]
        topic_vocab = ans[0][2]
        
        
elif model_spec['name'] == 'STM':
    if is_eval:
        pass
    else: 
        robjects.r.source('stm.R')
        ans = robjects.r.run_stm(docs = text_df,topic_n=model_spec['K'],max_itr=model_spec['epochs'],
                                 prevalence=model_spec['prevalence'],content=model_spec['content'],
                                 model_type = model_spec['pre_trained_model'],
                                 nits = model_spec['LDA_epochs'],burnin = model_spec['burnin_iteration'],
                                 save_flag = True)
        top_terms = np.asarray(ans[1]).reshape(model_spec['K'],50,order='F')
        topic_doc_dist = 0.00

R[write to console]: Loading required package: rJava

R[write to console]: Loading required package: mallet

R[write to console]: Loading required package: reshape2

R[write to console]: Loading required package: reticulate




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Loading required package: qdapTools

R[write to console]: Loading required package: reader

R[write to console]: Loading required package: NCmisc

R[write to console]: 
Attaching package: ‘reader’


R[write to console]: The following objects are masked from ‘package:NCmisc’:

    cat.path, get.ext, rmv.ext


R[write to console]: Loading required package: dplyr

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following object is masked from ‘package:qdapTools’:

    id


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: ggplot2

R[write to console]: Loading required package: jsonlite



[1] "Building mallet instance ..."
[1] "Building topic trainer ..."
[1] "Loading documents in mallet instance into topic trainer ..."
[1] "Training model ..."


Apr 16, 2023 10:28:57 AM cc.mallet.topics.ParallelTopicModel <init>
INFO: Mallet LDA: 10 topics, 4 topic bits, 1111 topic mask
Apr 16, 2023 10:28:57 AM cc.mallet.topics.ParallelTopicModel initializeHistograms
INFO: max tokens: 595
Apr 16, 2023 10:28:57 AM cc.mallet.topics.ParallelTopicModel initializeHistograms
INFO: total tokens: 36544
Apr 16, 2023 10:28:58 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <10> LL/token: -8.65356
Apr 16, 2023 10:28:58 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <20> LL/token: -8.43402
Apr 16, 2023 10:28:58 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <30> LL/token: -8.34735
Apr 16, 2023 10:28:58 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <40> LL/token: -8.29464
Apr 16, 2023 10:28:58 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: 
0	0.5	things try back love us sure make 
1	0.5	im like time get dont art trying 
2	0.5	enough would fucking even recovery every really 
3	0.5	day going take want something care make 
4

[1] "Training complete."


Apr 16, 2023 10:29:05 AM cc.mallet.topics.ParallelTopicModel optimizeBeta
INFO: [beta: 0.08131] 
Apr 16, 2023 10:29:05 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <1980> LL/token: -7.95804
Apr 16, 2023 10:29:05 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: <1990> LL/token: -7.95058
Apr 16, 2023 10:29:05 AM cc.mallet.topics.ParallelTopicModel estimate
INFO: 
0	0.07993	stay things mental let us illness gotta 
1	0.3057	im today day art good get got 
2	0.03681	would ask sick hair told better made 
3	0.111	actually today going day feel energy really 
4	0.079	life longer self nothing eating sad fit 
5	0.17991	eat food today still love eating foods 
6	0.09266	recovery tags people pro ed post ana 
7	0.08213	water drink protein try fruit coffee im 
8	0.072	body like feel look even great proud 
9	0.79595	recovery like get dont want time feel 

Apr 16, 2023 10:29:05 AM cc.mallet.topics.ParallelTopicModel optimizeBeta
INFO: [beta: 0.08185] 
Apr 16, 2023 10:29:05 AM cc.mallet.topi

# Post-processing

Running Coherence

In [6]:
from lda_mallet import lda_score
import pickle

# import importlib
# importlib.reload(scorer.score_per_topic(None))

#initiate scorer
scorer = lda_score(wiki_path='./data/wiki_sampled_5p.txt',db_path='./db/wiki_5p_old/')

'''
loading wikipedia vocab to 
    1. remove the top terms that are not in the wikipedia
    2. use for gensim.coherence vocab argument in case any pairs are not in the DB
'''
with open('./data/wiki_full_vocab.obj','rb') as f:
    scorer.wiki_vocab_dict = pickle.load(f)
    
#add all_top_terms (except the ones that are not in the wiki_pedia vocab)
scorer.all_top_terms = [[t for t in top_terms[i].split() if t in scorer.wiki_vocab_dict.token2id.keys()] for i in range(len(top_terms))]
#the output is a dictionary: the key is 0-n_topics and associated with npmi score for top-k topics 
#in which k=5,10,15,20
topic_scores = scorer.score_per_topic(None)
#getting avergae of multiple k for each topic
topic_score_avg = [np.mean([vi for ki,vi in v.items()]) for k,v in topic_scores.items()]

Load NPMI coherence DB. 
Number of keys : 365624


# Storing to DB

The process is described in the code cell below

**Used the link below to insert and load blob**

https://pynative.com/python-mysql-blob-insert-retrieve-file-image-as-a-blob-in-mysql/

In [7]:
#1. save model id (for LDA save as zip and then save)
'''
2. for each topic
    3. save topic
    4. retreive the saved topic_id and save r_topic_model
    5. for each doc save r_topic_post
'''
###table format
#model: model_id, data, model_type, last_used?
# topic: topic_id, coherence, term1-50, probab1-50
#r_topic_model: topic_id, model_id 
#r_topic_post: topic_id, post_id, proportion

##check the columns of a table
# cursor.execute("show columns from r_topic_post") #execute a command
# myresult = cursor.fetchall() #get all the results 
# myresult

'\n2. for each topic\n    3. save topic\n    4. retreive the saved topic_id and save r_topic_model\n    5. for each doc save r_topic_post\n'

In [8]:
#saving the model first
#model: model_id, data, model_type, last_used?
if model_spec['name'].lower() == 'lda':
    #read binary data of the saved model
    with open('MalletState_ '+model_spec['save_path']+' .gz','rb') as model_file:
        binary_data = model_file.read()
else:
    pass #write codes for other models

#storing the model
cursor.execute(" INSERT INTO model (data, model_type) VALUES (%s,%s)",
                    (binary_data,model_spec['name']))

In [9]:
#load the model with model id to use for next steps
cursor.execute("Select * from model ORDER BY model_id DESC LIMIT 1")
result = cursor.fetchall()
model_id,model_name = result[0][0],result[0][2]

In [10]:
#saving each topic separately alongside relation to model table and post table
for k in range(0,model_spec['K']):#iterating over each topic
    #getting topic top terms
    topic_top_terms = top_terms[k].split()
    #we have to find the index in vocab (ans[0][2]) and then call ans[0][1][k][found_index] to get
    #the probability of each term
    probab = [float(term_probab[k][i]) for w in topic_top_terms for i in range(len(topic_vocab)) if w == topic_vocab[i]]
    #adding coherence score, top terms and their probabilities
    vals = tuple([float(topic_score_avg[k])]) + tuple(topic_top_terms) + tuple(probab)
    #save each topic seperately
    cursor.execute(" INSERT INTO topic(coherence,{0},{1}) VALUES (%s,{2})".format(
                                        ','.join(['term'+str(i) for i in range(1,len(topic_top_terms)+1)]),
                                        ','.join(['prob'+str(i) for i in range(1,len(topic_top_terms)+1)]),
                                        ','.join(['%s']*(len(topic_top_terms)*2))),vals)
    
    '''
    retreiving saved topic_id and save an entry for topic_id,model_id in topic_model_r table
    '''
    #load the latest stored topic
    cursor.execute("Select * from topic ORDER BY topic_id DESC LIMIT 1")
    result = cursor.fetchall()
    topic_id = result[0][0]
    #save the topic_id,model_id in r_topic_model table 
    cursor.execute(" INSERT INTO r_topic_model(topic_id,model_id) VALUES (%s,%s)",
                  (topic_id,model_id,))
    
    '''
    saving topic-document proportion for each topic-doc pairs
    '''
    #r_topic_post : topic_id, post_id, proportion
    doc_counter = 0
    for doc_id in list(text_df.id):
        cursor.execute(" INSERT INTO r_topic_post(topic_id,post_id,proportion) VALUES (%s,%s,%s)",
                  (topic_id,doc_id,float(topic_doc_dist[doc_counter,k])))
        doc_counter += 1
        

In [19]:
# cursor.execute("delete from r_topic_model")
# cursor.execute("delete from r_topic_post")
# cursor.execute("delete from topic")
# cursor.execute("delete from model")

# cursor.execute("select * from topic")
# cursor.fetchall()